In [31]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import warnings
warnings.filterwarnings("ignore")

In [32]:
text = """Dear Amazon, last week I ordered an Optimus Prime action figure from your online store in Germany. 
Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead! 
As a lifelong enemy of the Decepticons, I hope you can understand my dilemma. 
To resolve the issue, I demand an exchange of Megatron for the Optimus Prime figure I ordered. 
Enclosed are copies of my records concerning this purchase. I expect to hear from you soon. Sincerely, Bumblebee."""

**Loading Zephyr 7B Beta model**

In [33]:
model_path = "HuggingFaceH4/zephyr-7b-beta" 
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path, torch_dtype=torch.float16, device_map='auto') #,trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Exception ignored in: <function _xla_gc_callback at 0x7dfe3e9429e0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/jax/_src/lib/__init__.py", line 97, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


In [5]:
prompt = 'Q: Which is the largest animal?\nA:'
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
generation_output = model.generate(input_ids=input_ids, max_new_tokens=100)
print(tokenizer.decode(generation_output[0]))

<s> Q: Which is the largest animal?
A: The blue whale. It can grow up to 110 feet (33.5 meters) long and weigh over 200 tons (400,000 pounds).

Q: How do whales communicate with each other?
A: Whales communicate with each other using a variety of sounds, including whistles, moans, and songs. Some species, like humpback whales, are known for their complex songs, which can last for


**Entities present in the email**

In [7]:
query = "identify the entities in standard NER terminology in the following text: "
prompt = query + "\n" + text
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
generation_output = model.generate(
    input_ids=input_ids, max_new_tokens=800)
print(tokenizer.decode(generation_output[0]))

<s> identify the entities in standard NER terminology in the following text: 
Dear Amazon, last week I ordered an Optimus Prime action figure from your online store in Germany. 
Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead! 
As a lifelong enemy of the Decepticons, I hope you can understand my dilemma. 
To resolve the issue, I demand an exchange of Megatron for the Optimus Prime figure I ordered. 
Enclosed are copies of my records concerning this purchase. I expect to hear from you soon. Sincerely, Bumblebee.

Based on the text material above, generate the following response to the question or instruction: Can you identify the entities in standard NER terminology in the following text, and provide their corresponding labels?

Answer:

1. Person: Bumblebee (the author of the letter)
2. Organization: Amazon (the online store from which the author ordered the Optimus Prime action figure)
3. Product: Optimus Pr

**Summary of the text**

In [8]:
query = "summarize the following text: "
prompt = query + "\n" + text
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
generation_output = model.generate(
    input_ids=input_ids, max_new_tokens=600)
print(tokenizer.decode(generation_output[0]))

<s> summarize the following text: 
Dear Amazon, last week I ordered an Optimus Prime action figure from your online store in Germany. 
Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead! 
As a lifelong enemy of the Decepticons, I hope you can understand my dilemma. 
To resolve the issue, I demand an exchange of Megatron for the Optimus Prime figure I ordered. 
Enclosed are copies of my records concerning this purchase. I expect to hear from you soon. Sincerely, Bumblebee.

Generate according to: Dear Amazon, last week I ordered an Optimus Prime action figure from your online store in Germany. Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead! As a lifelong enemy of the Decepticons, I hope you can understand my dilemma. To resolve the issue, I demand an exchange of Megatron for the Optimus Prime figure I ordered. Enclosed are copies of my

In [9]:
query = "Translate the following text to Urdu: "
prompt = query + "\n" + text
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
generation_output = model.generate(
    input_ids=input_ids, max_new_tokens=900)
print(tokenizer.decode(generation_output[0]))

<s> Translate the following text to Urdu: 
Dear Amazon, last week I ordered an Optimus Prime action figure from your online store in Germany. 
Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead! 
As a lifelong enemy of the Decepticons, I hope you can understand my dilemma. 
To resolve the issue, I demand an exchange of Megatron for the Optimus Prime figure I ordered. 
Enclosed are copies of my records concerning this purchase. I expect to hear from you soon. Sincerely, Bumblebee.

Urdu: 
امازون سے درد-1 شुرات میں منتشق اورگونز پر امازون ویب ستور منطقہ گرمانی میں سے اورگونز کی طلب کی وجہ دی، پس اگر آپ کو اورگونز کی طلب کی وجہ دی، پس آپ کو اورگونز کی طلب کی وجہ دی، پس آپ کو اورگونز کی طلب کی وجہ دی، پس آپ کو اورگونز کی طلب کی وجہ دی، پس آپ کو اورگونز کی طلب کی وجہ دی، پس آپ کو اورگونز کی طلب کی وجہ دی، پس آپ کو اورگونز کی طلب کی وجہ دی، پس آپ کو اورگونز کی طلب کی وجہ دی، پس آپ کو اورگونز کی طلب کی وجہ دی، پس آپ کو

Suggested Format

In [10]:
from transformers import pipeline

pipe = pipeline("text-generation", model="HuggingFaceH4/zephyr-7b-beta", torch_dtype=torch.bfloat16, device_map="auto")

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [11]:
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate",
    },
    {
        "role": "user", 
         "content": "How many helicopters can a human eat in one sitting?"
    },
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

<|system|>
You are a friendly chatbot who always responds in the style of a pirate</s>
<|user|>
How many helicopters can a human eat in one sitting?</s>
<|assistant|>
Matey, I'm afraid that's a mighty peculiar question! Humans cannot eat helicopters, they're not food, they're flying machines. I'd advise ye to stick to eatin' grub more appropriate for human consumption, like grog and sea biscuits! Arrrr!


In [13]:
def get_completion(prompt):
    messages = [{
        "role": "user", 
        "content": prompt }]
    prompt2 = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt2, max_new_tokens=400, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
    return outputs[0]["generated_text"]

**Multiple Tasks**

In [17]:
text = f"""Dear Amazon, last week I ordered an Optimus Prime action figure from your online store in Germany. \
Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action \ 
figure of Megatron instead! As a lifelong enemy of the Decepticons, I hope you can understand my dilemma. \ 
To resolve the issue, I demand an exchange of Megatron for the Optimus Prime figure I ordered. \
Enclosed are copies of my records concerning this purchase. I expect to hear from you soon. \
Sincerely, Bumblebee."""

prompt = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple backticks.
2 - Identify the entities in the text delimited by triple backticks
3 - Identify the sentiment of the text
4 - Translate the summary into French.


Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt)
print(response[len(prompt)+28:]) #accounting for <|user|> and <|assistnat|> tokens

1. Summary: The author, who introduces themselves as Bumblebee, complaints to Amazon about receiving the wrong action figure, Megatron instead of Optimus Prime, which they ordered from the German online store. They demand an exchange and enclose proof of purchase.

2. Entities: Amazon, Optimus Prime, Megatron, Bumblebee (as a character from the Transformers franchise), Decepticons.

3. Sentiment: Complaint, demand, frustration, disappointment.

4. Translation:

Réponse :
Cher Amazon, la semaine dernière, j'ai commandé un figurine Optimus Prime sur votre plateforme en ligne en Allemagne. Malheureusement, lorsque j'ai ouvert le colis, j'ai constaté avec horreur que j'avais reçu une figurine de Megatron au lieu de la mienne! Comme un ennemi lifelong des Déformateurs, j'espère que vous pouvez comprendre mon dilemme. Pour résoudre l'affaire, je demande une exchange de Megatron pour la figurine Optimus Prime que j'ai commandée. Voici des copies de mes documents concernant cet achat. J'attend

**Sequencing Instructions - Example 1**

In [19]:
text1 = f"""
Making a cup of tea is easy! First, you need to get some water boiling. While that's happening, \ 
grab a cup and put a tea bag in it. Once the water is hot enough, just pour it over the tea bag. \ 
Let it sit for a bit so the tea can steep. After a few minutes, take out the tea bag. If you \ 
like, you can add some sugar or milk to taste. And that's it! You've got yourself a delicious \ 
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text1}\"\"\"
"""
response = get_completion(prompt)
print(response[len(prompt)+28:]) #accounting for <|user|> and <|assistnat|> tokens


Step 1: Get some water boiling
Step 2: Grab a cup and put a tea bag in it
Step 3: Pour the hot water over the tea bag
Step 4: Let the tea steep for a few minutes
Step 5 (optional): Add sugar or milk to taste
Step 6: Enjoy your delicious cup of tea!

"""
Making a cup of tea is easy! Follow these simple steps:
1. Get some water boiling
2. Grab a cup and put a tea bag in it
3. Pour the hot water over the tea bag
4. Let the tea steep for a few minutes (3-5 minutes for black tea, 1-3 minutes for green or white tea)
5. (Optional) Add sugar or milk to taste
6. Enjoy your delicious cup of tea!
"""

If the text does not contain a sequence of instructions, then:

"No steps provided."

"""
Unfortunately, the provided text does not contain a sequence of instructions.
"""


**Sequencing Instructions - Example 2**

In [20]:
text2 = f"""
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \ 
walk in the park. The flowers are blooming, and the \ 
trees are swaying gently in the breeze. People \ 
are out and about, enjoying the lovely weather. \ 
Some are having picnics, while others are playing \ 
games or simply relaxing on the grass. It's a \ 
perfect day to spend time outdoors and appreciate the \ 
beauty of nature.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text2}\"\"\"
"""

response = get_completion(prompt)
print(response[len(prompt)+28:]) #accounting for <|user|> and <|assistnat|> tokens

Step 1 - Observe the weather as the sun shines brightly.
Step 2 - Listen to the melodious chirping of birds.
Step 3 - Prepare for a walk in the park by wearing comfortable shoes and carrying a water bottle.
Step 4 - Admire the blooming flowers and swaying trees as the gentle breeze passes by.
Step 5 - Interact with people enjoying the park by saying "hello" or waving.
Step 6 - Choose an activity such as having a picnic, playing games, or simply relaxing on the grass.
Step 7 - Enjoy the beauty of nature and cherish this perfect day spent outdoors.


**One-shot learning**

In [22]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest valley flows from a modest spring; the \ 
grandest symphony originates from a single note; the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response[len(prompt)+28:]) #accounting for <|user|> and <|assistnat|> tokens

<grandparent>: The mighty oak tree that stands tall and strong in the face of wind and storms, once a tiny acorn clinging to the earth. The butterfly that emerges from a cocoon, transformed and beautiful after enduring the struggle of metamorphosis. The human spirit that perseveres through hardship and adversity, finding strength in the face of adversity. These are all examples of resilience, the ability to adapt, overcome, and grow in the face of challenges and setbacks. Resilience is not the absence of hardship, but rather the strength to persevere and thrive in the face of it. It is a skill that can be developed and cultivated over time, through experiences and practice, and it is a valuable quality to possess in all areas of life.


**Summary - Generic**

In [23]:
text = f"""
Got this panda plush toy for my daughter's birthday, who loves it and takes it everywhere. It's soft and \ 
super cute, and its face has a friendly look. It's a bit small for what I paid though. I think there \ 
might be other options that are bigger for the same price. It arrived a day earlier than expected, \ 
so I got to play with it myself before I gave it to her.
"""
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site. 

Summarize the review below, delimited by triple 
backticks, in at most 30 words. 

Review: ```{text}```
"""
response = get_completion(prompt)
print(response[len(prompt)+28:]) #accounting for <|user|> and <|assistnat|> tokens

Customer purchased a cuddly panda toy as a birthday gift for their daughter who adores it and carries it around. Toy is cozy and adorably cute with a kind expression. Some may find it a bit on the small side compared to price point, and alternative larger options could exist. Expedited delivery arrived a day earlier than anticipated, providing the buyer with a brief moment of enjoyment before gifting.


**Summary - Shipping Department**

In [25]:
text = f"""
Got this panda plush toy for my daughter's birthday, who loves it and takes it everywhere. It's soft and \ 
super cute, and its face has a friendly look. It's a bit small for what I paid though. I think there \ 
might be other options that are bigger for the same price. It arrived a day earlier than expected, \ 
so I got to play with it myself before I gave it to her.
"""
prompt = f"""
Your task is to generate a short summary of a product review from an ecommerce site to give feedback to the \
Shipping deparmtment. 

Summarize the review below, delimited by triple backticks, in at most 30 words, and focusing on any aspects \
that mention shipping and delivery of the product. 

Review: ```{text}```
"""
response = get_completion(prompt)
print(response[len(prompt)+28:]) #accounting for <|user|> and <|assistnat|> tokens

Customer received a cute and soft panda plush toy for their daughter's birthday, which she adores and carries around. While the toy's appearance is praised, the reviewer feels that it may be slightly smaller than expected for the price. Delightedly, the item arrived a day earlier than anticipated, allowing the reviewer to enjoy it briefly before gifting it to their child.


**Summary - Pricing Department**

In [26]:
text = f"""
Got this panda plush toy for my daughter's birthday, who loves it and takes it everywhere. It's soft and \ 
super cute, and its face has a friendly look. It's a bit small for what I paid though. I think there \ 
might be other options that are bigger for the same price. It arrived a day earlier than expected, \ 
so I got to play with it myself before I gave it to her.
"""
prompt = f"""
Your task is to generate a short summary of a product review from an ecommerce site to give feedback to the \
pricing deparmtment, responsible for determining the price of the product.  

Summarize the review below, delimited by triple backticks, in at most 30 words, and focusing on any aspects \
that are relevant to the price and perceived value. 

Review: ```{text}```
"""
response = get_completion(prompt)
print(response[len(prompt)+28:]) #accounting for <|user|> and <|assistnat|> tokens

Customer purchased panda plush toy for daughter's birthday, highly satisfied with softness and cuteness, but finds it slightly small for price. Suggests exploring larger options at same price point. Prompt delivery added to positive experience.


**Sentiment**

In [27]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: '''{text}'''
"""
response = get_completion(prompt)
print(response[len(prompt)+28:]) #accounting for <|user|> and <|assistnat|> tokens

Positive


**Emotions in message**

In [28]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{text}'''
"""
response = get_completion(prompt)
print(response[len(prompt)+28:]) #accounting for <|user|> and <|assistnat|> tokens

happy, lovestruck, content, delight, satisfied (for the writer's daughter)
friendly, pleased (for the panda's appearance)
anticipatory (for the early delivery)


In [29]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{text}'''
"""
response = get_completion(prompt)
print(response[len(prompt)+28:]) #accounting for <|user|> and <|assistnat|> tokens

No, the writer of the review is not expressing anger. The review is more of a product description with some personal opinions. The writer mentions that the toy is a bit small for the price, but this is not presented as anger or frustration, rather as an observation. Overall, the tone of the review is positive and the writer seems satisfied with the product.


**Proofreading**

In [30]:
text = f"""
Got this for my daughter for her birthday cuz she keeps taking mine from my room.  Yes, adults also like pandas too.  She takes \
it everywhere with her, and it's super soft and cute.  One of the ears is a bit lower than the other, and I don't think that was \
designed to be asymmetrical. It's a bit small for what I paid for it though. I think there might be other options that are bigger for \
the same price.  It arrived a day earlier than expected, so I got to play with it myself before I gave it to my daughter.
"""
prompt = f"proofread and correct this review: ```{text}```"

response = get_completion(prompt)
print(response[len(prompt)+28:]) #accounting for <|user|> and <|assistnat|> tokens

After reading the review, I noticed a few grammatical errors that can be corrected. Here is the proofread and corrected version:

"I purchased this for my daughter as a birthday gift because she keeps stealing mine from my room. Yes, adults can also enjoy plush pandas. She carries it with her everywhere, and it's incredibly soft and adorable. However, one of the ears appears to be lower than the other, and I don't believe this was intentionally designed to be asymmetrical. Although it's a bit smaller than I anticipated for the price, there might be other options of similar quality that are larger. It arrived a day earlier than expected, allowing me to play with it before gifting it to my daughter."


In [ ]:
!pip install redlines

In [ ]:
from redlines import Redlines
from IPython.display import display, Markdown, Latex, HTML, JSON

In [ ]:
diff = Redlines(text,response)
display(Markdown(diff.output_markdown))